In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units
from astropy.constants import G
import importlib

import dataset_compute
import snapshot_obj

In [ ]:
importlib.reload(dataset_compute)
importlib.reload(snapshot_obj)

## Construct data dictionary

Add entries for each simulation, and specify M31 and MW galaxies:

In [ ]:
snap_id = 127
sim_ids = ["V1_MR_fix", "V1_MR_curvaton_p082_fix"]
names = ["plain-LCDM", "spec-p082"]
direc = "MediumResolution"

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = "by_r"

Read datasets, split into satellites and isolated galaxies, and add to the data dictionary. We also disregard dark halos and potential spurious halos with $v_\mathrm{max} = 0$.

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data["snapshot"]
    cops = snap.get_subhalos("CentreOfPotential") * units.cm.to(units.Mpc)
    mass = snap.get_subhalos("Mass") * units.g.to(units.Msun)

    # Split into satellites:    
    m31 = sim_data["M31_identifier"]
    mw = sim_data["MW_identifier"]
    if distinction == "by_r":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, m31, mw)
    elif distinction == "by_gn":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, m31, mw)
        
    # Compute masking arrays:
    mask_m31 = masks_sat[0]
    mask_mw = masks_sat[1]
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    
    # Prune potential spurious:
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    mask_m31 = mask_m31[mask_nonzero_vmax]
    mask_mw = mask_mw[mask_nonzero_vmax]
    mask_lum = mask_lum[mask_nonzero_vmax]
    mask_dark = mask_dark[mask_nonzero_vmax]

    # Add datasets to dictionary:
    data[name]["LG_centre"] = dataset_compute.compute_LG_centre(snap, m31, mw) \
        * units.cm.to(units.Mpc)
    m31_idx = snap.index_of_halo(m31[0], m31[1])
    data[name]["M31"] = {"COP": cops[m31_idx], "Mass": mass[m31_idx]}
    mw_idx = snap.index_of_halo(mw[0], mw[1])
    data[name]["MW"] = {"COP": cops[mw_idx], "Mass": mass[mw_idx]}
    data[name]["COP"] = cops
    data[name]["Mass"] = mass
    
    # Add selections (masking arrays):
    data[name]["Selections"] = {
        "M31": mask_m31,
        "MW": mask_mw,
        "Isolated": mask_isol,
        "Luminous": mask_lum,
        "Dark": mask_dark
    }

In [ ]:
# Construct saving location:
filename = "Satellite_locations_Seminar"
for name in names:
    filename += "_{}".format(name)
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures", direc)
filename = os.path.join(path, filename)

In [ ]:
projs = [["x", "y", 0, 1],
         ["x", "z", 0, 2]]

fig, axes = plt.subplots(sharey='all', sharex='all', figsize=(8,8),
                         ncols=len(data), nrows=len(projs))
plt.subplots_adjust(wspace=0.04)
plt.subplots_adjust(hspace=0.04)

axes[0,0].set_ylabel("$y [\mathrm{Mpc}]$")
axes[1,0].set_xlabel("$x [\mathrm{Mpc}]$")
axes[1,0].set_ylabel("$y [\mathrm{Mpc}]$")
axes[1,1].set_xlabel("$x [\mathrm{Mpc}]$")

# Get average of LG centre across simulations:
centre = np.mean(np.vstack(
    [sim_data["LG_centre"] for sim_data in data.values()]),
                 axis=0)
  
# Set axis limits:
ax_size = 5
axes[1, 0].set_xlim(centre[0] - ax_size/2, 
                    centre[0] + ax_size/2)
axes[1, 0].set_ylim(centre[1] - ax_size/2, 
                    centre[1] + ax_size/2)

# Iterate through simulations (columns):
for i, (key, sim_data) in enumerate(data.items()):
    
    axes[0, i].set_title(key)
    
    # Set marker size limits:
    smin = 0.001; smax = 3
    all_dwarfs = sim_data["Mass"][
        np.logical_or.reduce([sim_data["Selections"]["M31"],
                               sim_data["Selections"]["MW"],
                               sim_data["Selections"]["Isolated"]])
    ]
    mmax = np.min(all_dwarfs)
    mmin = np.max(all_dwarfs)
    
    # For vmax:
    def mass_to_marker(x):
        return 1/(mmax-mmin) * (smin*(x-mmin) - smax*(x-mmax))
    
    # Set marker transparency:
    a = 0.7
    
    # Plot Dark on the upper row:
    mask_isol = np.logical_not(
        np.logical_or(sim_data["Selections"]["M31"], 
                      sim_data["Selections"]["MW"])
    )
    axes[0,i].scatter(sim_data["COP"][mask_isol][:,0], 
                      sim_data["COP"][mask_isol][:,1],
                      c='gray',
                      alpha=a,
                      s=mass_to_marker(sim_data["Mass"][mask_isol]))
                              
    mask_m31 = sim_data["Selections"]["M31"]
    axes[0,i].scatter(sim_data["COP"][mask_m31][:,0], 
                      sim_data["COP"][mask_m31][:,1],
                      c='pink',
                      alpha=a,
                      s=mass_to_marker(sim_data["Mass"][mask_m31]))
    mask_mw = sim_data["Selections"]["MW"]
    axes[0,i].scatter(sim_data["COP"][mask_mw][:,0], 
                      sim_data["COP"][mask_mw][:,1],
                      c='lightblue', 
                      alpha=a,
                      s=mass_to_marker(sim_data["Mass"][mask_mw]))
    
    axes[0,i].scatter(sim_data["M31"]["COP"][0], sim_data["M31"]["COP"][1],
                      c='red',
                      s=4)
    axes[0,i].scatter(sim_data["MW"]["COP"][0], sim_data["MW"]["COP"][1],
                      c='blue',
                      s=4)
        
    # Plot Luminous on the lower:
    mask_isol = np.logical_and(
        sim_data["Selections"]["Luminous"],
        np.logical_not(np.logical_or(sim_data["Selections"]["M31"], 
                                     sim_data["Selections"]["MW"]))
    )
    axes[1,i].scatter(sim_data["COP"][mask_isol][:,0], 
                      sim_data["COP"][mask_isol][:,1],
                      facecolors="none", 
                      edgecolors="gray",
                      s=10*mass_to_marker(sim_data["Mass"][mask_isol]))
                              
    mask_m31 = np.logical_and(sim_data["Selections"]["Luminous"],
                              sim_data["Selections"]["M31"])
    axes[1,i].scatter(sim_data["COP"][mask_m31][:,0], 
                      sim_data["COP"][mask_m31][:,1],
                      facecolors="none", 
                      edgecolors='pink',
                      alpha=a,
                      s=10*mass_to_marker(sim_data["Mass"][mask_m31]))
    mask_mw = np.logical_and(sim_data["Selections"]["Luminous"],
                             sim_data["Selections"]["MW"])
    axes[1,i].scatter(sim_data["COP"][mask_mw][:,0], 
                      sim_data["COP"][mask_mw][:,1],
                      facecolors="none", 
                      edgecolors='lightblue',
                      alpha=a,
                      s=10*mass_to_marker(sim_data["Mass"][mask_mw]))
    
    axes[1,i].scatter(sim_data["M31"]["COP"][0], sim_data["M31"]["COP"][1],
                      c='red',
                      s=4)
    axes[1,i].scatter(sim_data["MW"]["COP"][0], sim_data["MW"]["COP"][1],
                      c='blue',
                      s=4)
    
    # Set text size:
    s = 14
    axes[0, i].title.set_size(s)
    axes[1, i].xaxis.label.set_size(s)
    axes[0, 0].yaxis.label.set_size(s)
    axes[1, 0].yaxis.label.set_size(s)
    
    
    # Plot circles:
    def circle(centre_x, centre_y, r, n):
        t = 4*np.pi/n * np.arange(n/2)
        x = centre_x + r * np.cos(t)
        y = centre_y + r * np.sin(t)
        return x, y

    lw = 0.3
    x_circ, y_circ = circle(centre[0], centre[1], 2, 10000)
    axes[0,i].plot(x_circ, y_circ, c='black', linestyle='dashed', linewidth=lw)
    axes[1,i].plot(x_circ, y_circ, c='black', linestyle='dashed', linewidth=lw)
    
# plt.savefig(filename, dpi=300)